In [ ]:
# This Notebook is indended to accompany our original calculations in PyFunding_1.
# In this Notebook, we are looking to enhance and optimize the previous model.
# We are making alterations to both our data and training methods.  Check the markdown text, as we will call out when these changes occur.

## Preprocessing

In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID column, 'EIN'.
# Note that this is a change from the previous model, where we omitted 'Name' as well.
application_df = application_df.drop(["EIN"], 1)
application_df.head()

<ipython-input-26-48a3af689036>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN"], 1)


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [28]:
# Finding and singling out names with a count greater than 5
name_counts = application_df["NAME"].value_counts()
name_counts[name_counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [31]:
# For name counts less than 5, we are renaming them to "Other."
# Giving ourselves more columns of data to work with.
names_to_replace = list(name_counts[name_counts<=5].index)
for app in names_to_replace:
  application_df["NAME"] = application_df["NAME"].replace(app, "Other")
application_df["NAME"].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [32]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                       355
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [33]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [35]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [38]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [39]:
# Classification Counts Greater than 10
class_counts[class_counts > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [40]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [41]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(application_df)
app_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [42]:
# Split our preprocessed data into our features and target arrays
X = app_dummies_df.drop(["IS_SUCCESSFUL"], axis="columns").values
y = app_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=78)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# For Enhancement and Optimization:
  # Moving hidden_nodes1 to 100 from 80
  # Adding hidden_nodes3 at 10

input_feature_count = len(X_train[0])
hidden_nodes1 = 100
hidden_nodes2 = 30
hidden_nodes3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer, using Relu Activation
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_feature_count, activation="relu")
)

# Second hidden layer, using Sigmoid Activation
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes2, input_dim=input_feature_count, activation="sigmoid")
)

# Third hidden layer, using Sigmoid Activation
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes3, input_dim=input_feature_count, activation="sigmoid")
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               39900     
                                                                 
 dense_9 (Dense)             (None, 30)                3030      
                                                                 
 dense_10 (Dense)            (None, 10)                310       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 43251 (168.95 KB)
Trainable params: 43251 (168.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5055 - accuracy: 0.7621
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4453 - accuracy: 0.7928
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4360 - accuracy: 0.7948
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4312 - accuracy: 0.7956
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4285 - accuracy: 0.7949
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4256 - accuracy: 0.7977
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4242 - accuracy: 0.7982
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4226 - accuracy: 0.7991
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4218 - accuracy: 0.7986
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4206 - accura

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4521 - accuracy: 0.7879 - 453ms/epoch - 2ms/step
Loss: 0.45206063985824585, Accuracy: 0.7878717184066772


In [48]:
# Export our model to HDF5 file
nn.save("PyFunding_2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Overview of the analysis:**

**Data Preprocessing**
1. What variable(s) are the target(s) for your model?
  * The variables that were targeted were in the "IS_SUCCESSFUL" column.

2. What variable(s) are the features for your model?
  * The first model uses the following features:
    * Application Type
    * Affiliation Classification
    * Use Case
    * Organization
    * Income
    * Special Considerations
    * Status
    * Ask Amount
  * The second model uses the following features:
    * Name (but broken out into multiple columns for the most common names)
    * Application Type
    * Affiliation Classification
    * Use Case
    * Organization
    * Income
    * Special Considerations
    * Status
    * Ask Amount

3. What variable(s) should be removed from the input data because they are neither targets nor features?
  * In the first model, we removed the Employee Identification Number (EIN) and Name as features as prompted.
  * In the second model, we kept EIN removed, but re-introduced Name as a feature, and used it as a basis for comparison.
    
**Compiling, Training, and Evaluating the Model**
1. How many neurons, layers, and activation functions did you select for your neural network model, and why?
  * In the first model, we used two hidden layers:
    * The first hidden layer was given 80 nodes, and used Relu activation.
    * The second hidden layer was given 30 nodes, and also used Relu activation.
    
    The number of nodes for the first model was chosen due to the size of the dataset.  33k rows and 12 columns was a decently large set of data, so a node size of 80 seemed on the high end, but appropriate. The 30 nodes for the second hidden layer would then be able to suss out the findings of the first. Relu activation was chosen as a default method to see if we could improve with Sigmoid in the second model.

  * In the second model, we used three hidden layers:
    * The first hidden layer was given 100 nodes, activation remained at relu.
    * The second hidden layer remained at 80 nodes, activation changed to Sigmoid.
    * The third hidden layer was given 10 nodes, Sigmoid activation was used.

    For the second model, we increased the number of nodes for the first hidden layer and changed the subsequent layers to Sigmoid, essentially giving additional passes to the data to see if anything new could be squeezed out.


2. Were you able to achieve the target model performance?
  * The first model did not achieve the desired performance target of 75%.  The results of the first model were 72.68% accuracy with 56.60% data loss.
  * However, the second model did meet the performance expectations, as it achieved an accuracy of 78.78% and the Loss went down to 45.20%.  

3. What steps did you take in your attempts to increase model performance?
  * We gave greater definition to our data by providing additional variables by splitting the Name column into multiple columns.  This gave the model more data to work with.  
  * In the second model, we upped the number of nodes in the first layer to 100, from 80.
  * We changed the second hidden layer's activation type to Sigmoid and added a third hidden layer with 10 nodes, which also used Sigmoid.
  
**Summary:**
Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.
  * We were able to come to some reasonbly good results in our second model.  The model exceeded the 75% requirement, and our data loss also improved.  
  
  * Based on these predictions, we have a reasonable expectation that if an applicant has been through the process five times or more, they are likely to be successful in being approved for funding.  However, they should also be aware that the amount requested is also a factor in these decisions and should try to remain in the funding guidelines set out by the application types T3, T4, T5, T6, T7, T8, T10, and T19.
  
  * Another option for making predictions for this kind of dataset might be RandomForest.  In the cells below, we ran our data through the RandomForest model and came up with similar accuracy scores.  The first model gave a 71.1%, and the second model gave 77.6%.  This tells us that if we want to improve our results, the solution would most likely be found in our data.

  * Another piece of data that might be helpful in helping us predict success or failure might be the dates of the application.  If an applicant is re-submitting too soon after receiving funding or being denied, could that factor in?  We may also want to consider what our budget allowances were at the time of application, does that factor in to a denial or acceptance? Bringing in those couple factors could help us make better predictions.

In [49]:
# Testing a RandomForest model with our data.
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [50]:
# Testing a RandomForest model with our data.
rf_model = RandomForestClassifier(n_estimators=120, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)
print(f"Accuracy Per Random Forest:  {accuracy_score(y_test, y_pred)}")

Accuracy Per Random Forest:  0.7764431486880466
